In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(os.path.dirname(os.getcwd()))
print(os.getcwd())

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


from resources.constants import *

pictures_df = pd.read_csv(PICTURE_TRIPLETS_CSV_PATH, sep=CSV_SEPARATOR)
outfits_df = pd.read_csv(OUTFITS_CSV_PATH, sep=CSV_SEPARATOR)
user_triplets_df = pd.read_csv(USER_ACTIVITY_TRIPLETS_CSV_PATH, sep=CSV_SEPARATOR)
# Ensure tags are lists
outfits_df["tag_categories"] = outfits_df["tag_categories"].apply(eval)
outfits_df["outfit_tags"] = outfits_df["outfit_tags"].apply(eval)


c:\Users\kaborg15\Python_projects\Vibrent_Dataset_Collection


In [2]:
import src.load_baseline_resources
import pickle
from resources.constants import EMBEDDING_MODEL_DICT_PICKLE_PATH
# loaded_embeddings_dict = src.load_baseline_resources.load_embeddings_form_folder()
# pickle.dump(loaded_embeddings_dict, open(EMBEDDING_MODEL_DICT_PICKLE_PATH, "wb"))

# Loading embeddings is expensive, so we save them to a pickle file
loaded_embeddings_dict = pickle.load(open(EMBEDDING_MODEL_DICT_PICKLE_PATH, "rb"))


pictures_df["embeddings"] = pictures_df["picture.id"].map(loaded_embeddings_dict)
outfit_pictures_df = pictures_df.groupby("outfit.id").agg({"picture.id": list, "embeddings": list}).reset_index()
outfits_df["embeddings"] = outfits_df["id"].map(outfit_pictures_df.set_index("outfit.id")["embeddings"])
na_embedding_outfit_ids = outfits_df[outfits_df["embeddings"].isna()]["id"]
outfits_df = outfits_df.dropna(subset=["embeddings"])

In [3]:
# Introduce group to rental triplets
id_group_dict = outfits_df[["id", "group"]].to_dict(orient="records")
id_group_dict = {x["id"]: x["group"] for x in id_group_dict}
user_triplets_df["group"] = user_triplets_df["outfit.id"].map(id_group_dict)
# Remove triplets with no embeddings
user_triplets_df = user_triplets_df[~user_triplets_df["outfit.id"].isin(na_embedding_outfit_ids)]

In [6]:
from src.prepare_train_test_splits import convert_user_orders_to_train_test_splits
user_orders_df = user_triplets_df.groupby("customer.id").agg({"outfit.id": list, "group":list, "meta.validFrom":list, "derived.bookingTime":list}).reset_index()
user_orders_df["num_orders"] = user_orders_df["outfit.id"].apply(lambda x: len(x))
user_orders_df = user_orders_df[user_orders_df["num_orders"] > 1]

user_splits_df, user_splits_unique_df = convert_user_orders_to_train_test_splits(user_orders_df)

No unique outfit found with groups ['group.8abe6af9eccc8b578c2ef59628f8b454'
 'group.96f4cce22d4a236e0652c67fc9b18d12'
 'group.8abe6af9eccc8b578c2ef59628f8b454'
 'group.96f4cce22d4a236e0652c67fc9b18d12']


In [ ]:
from tqdm.notebook import tqdm

def build_tag_dict(tags, tag_categories):
    tag_dict = {}
    for tag, tag_category in zip(tags, tag_categories):
        if tag_category not in tag_dict:
            tag_dict[tag_category] = []
        tag_dict[tag_category].append(tag)
    return tag_dict

tqdm.pandas()

outfits_df["tag_dict"] = outfits_df.progress_apply(lambda x: build_tag_dict(x["outfit_tags"], x["tag_categories"]), axis=1)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

all_tags = outfits_df["outfit_tags"].values.tolist()
mlb = MultiLabelBinarizer()
one_hot_encoded = mlb.fit_transform(all_tags)

In [ ]:
outfits_df["tag_dict"]

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
from tqdm.notebook import tqdm

NUM_ITEMS = 100

def find_rental_history_embeddings(outfit_ids, outfit_to_embedding_dict):
    return [outfit_to_embedding_dict[outfit_id] for outfit_id in outfit_ids]

def get_mean_embedding(embeddings):
    embeddings = np.array(embeddings)
    return np.mean(embeddings, axis=0)

def get_nearest_neighbors_batch(embeddings, nn, num_items, index_to_id):
    distances, indices = nn.kneighbors(embeddings, n_neighbors=num_items+1)
    ids = [[index_to_id[i] for i in idx[1:]] for idx in indices]
    distances = [dist[1:] for dist in distances]
    return ids, distances

def predict_nearest_neighbors(df, outfits_df, subset_length=-1):
    outfits_df["mean_embeddings"] = outfits_df["embeddings"].apply(lambda x: get_mean_embedding(x))
    outfit_to_embedding_dict = outfits_df.set_index("id")["mean_embeddings"].to_dict()
    index_to_outfit_dict = {i: outfit_id for i, outfit_id in enumerate(outfits_df["id"].values)}
    group_to_embedding_dict = outfits_df.set_index("group")["mean_embeddings"].to_dict()
    index_to_group_dict = {i: group for i, group in enumerate(outfits_df["group"].values)}

    df["train_id_embeddings"] = df["train_outfit_ids"].apply(lambda x: find_rental_history_embeddings(x, outfit_to_embedding_dict))
    df["train_group_embeddings"] = df["train_group"].apply(lambda x: find_rental_history_embeddings(x, group_to_embedding_dict))

    df["rental_history_id_embedding"] = df["train_id_embeddings"].apply(lambda x: get_mean_embedding(x))
    df["rental_history_group_embedding"] = df["train_group_embeddings"].apply(lambda x: get_mean_embedding(x))

    nearest_neighbors = NearestNeighbors(n_neighbors=NUM_ITEMS+1, metric="cosine")
    embeddings = np.stack(outfits_df["mean_embeddings"].values)
    nearest_neighbors.fit(embeddings)

    id_embeddings = np.stack(df["rental_history_id_embedding"].values)
    group_embeddings = np.stack(df["rental_history_group_embedding"].values)

    id_predictions, id_distances = get_nearest_neighbors_batch(id_embeddings, nearest_neighbors, NUM_ITEMS, index_to_outfit_dict)
    group_predictions, group_distances = get_nearest_neighbors_batch(group_embeddings, nearest_neighbors, NUM_ITEMS, index_to_group_dict)

    df["id_prediction"], df["id_prediction_distances"] = id_predictions, id_distances
    df["group_prediction"], df["group_prediction_distances"] = group_predictions, group_distances
    
    return df

# Apply to dataframes
tqdm.pandas()
user_splits_df = predict_nearest_neighbors(user_splits_df, outfits_df, subset_length=-1)
user_splits_unique_df = predict_nearest_neighbors(user_splits_unique_df, outfits_df, subset_length=-1)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Perform t-SNE
tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(embeddings)

# Plot the t-SNE result
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], s=5, cmap='viridis')
plt.title('t-SNE of Embeddings')
plt.xlabel('t-SNE component 1')
plt.ylabel('t-SNE component 2')
plt.show()

In [ ]:
# Test if test outfit is in train outfits for recommendation of repeated outfits
# def check_if_test_in_train(train_outfit_ids, test_outfit_id):
#     return test_outfit_id in train_outfit_ids

# user_splits_df["test_in_train"] = user_splits_df.apply(lambda x: check_if_test_in_train(x["train_outfit_ids"], x["test_outfit_id"]), axis=1)
# user_splits_unique_df["test_in_train"] = user_splits_unique_df.apply(lambda x: check_if_test_in_train(x["train_outfit_ids"], x["test_outfit_id"]), axis=1)
# display(user_splits_df["test_in_train"].value_counts())
# user_splits_df[["train_outfit_ids", "test_outfit_id", "id_prediction", "test_in_train"]]
# display(user_splits_unique_df["test_in_train"].value_counts())
# user_splits_unique_df[["train_outfit_ids", "test_outfit_id", "id_prediction", "test_in_train"]]


# user_splits_unique_df["train_length"] = user_splits_unique_df["train_outfit_ids"].apply(len)
# user_splits_unique_df["prediction_length"] = user_splits_unique_df["id_prediction"].apply(len)

In [ ]:
from IPython.display import display

def evaluate_hit_rate_at_n(test_id, predicted_ids, n=10):
    if predicted_ids is np.nan:
        print(f"None prediction for {test_id}!")
        return 0
    predicted_ids = predicted_ids[:n]
    if test_id in predicted_ids:
        #print(f"Hit at {n} for {test_id} in {predicted_ids}")
        return 1
    return 0

HIT_RATE_COLUMNS = ["id_hit_rate_at_100", "id_hit_rate_at_10", "group_hit_rate_at_100", "group_hit_rate_at_10"]
def evaluate_df_hit_rate_at_n(df, n=10):
    df["id_hit_rate_at_100"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_outfit_id"], x["id_prediction"], n=100), axis=1)
    df["id_hit_rate_at_10"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_outfit_id"], x["id_prediction"], n=10), axis=1)
    df["group_hit_rate_at_100"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_group"], x["group_prediction"], n=100), axis=1)
    df["group_hit_rate_at_10"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_group"], x["group_prediction"], n=10), axis=1)
    display(df[HIT_RATE_COLUMNS].mean())
    return df


user_splits_df = evaluate_df_hit_rate_at_n(user_splits_df, n=10)
user_splits_unique_df = evaluate_df_hit_rate_at_n(user_splits_unique_df, n=10)

In [ ]:
import pyperclip

def format_dicts_into_latex(all_dict, ind_dict, precision=4, run_name="Random"):
    first_row = f"{run_name} Ind: & {all_dict['id_hit_rate_at_10']:.{precision}f} & {all_dict['id_hit_rate_at_100']:.{precision}f} & {ind_dict['id_hit_rate_at_10']:.{precision}f} & {ind_dict['id_hit_rate_at_100']:.{precision}f} \\\\"
    second_row = f"{run_name} Groups: & {all_dict['group_hit_rate_at_10']:.{precision}f} & {all_dict['group_hit_rate_at_100']:.{precision}f} & {ind_dict['group_hit_rate_at_10']:.{precision}f} & {ind_dict['group_hit_rate_at_100']:.{precision}f} \\\\\\hline"
    full_string = first_row + "\n" + second_row + "\n"
    print(full_string)
    pyperclip.copy(full_string)

all_dict = {column: user_splits_df[column].mean() for column in HIT_RATE_COLUMNS}
ind_dict = {column: user_splits_unique_df[column].mean() for column in HIT_RATE_COLUMNS}

format_dicts_into_latex(all_dict, ind_dict, precision=4, run_name="Image Embeddings")

In [ ]:
import numpy as np

def get_outfit_category(tag_categories, tags, category):
    tag_categories, tags = np.array(tag_categories), np.array(tags)
    category_indexes = np.where(tag_categories == category)[0]
    if len(category_indexes) == 0:
        return ""
    cat_tags = tags[category_indexes]
    output = str(cat_tags[0])
    return output

outfits_df["size"] = outfits_df.apply(lambda x: get_outfit_category(x["tag_categories"], x["outfit_tags"], "Size"), axis=1)


In [ ]:
outfits_df